In [76]:
import thulac
import os
import re
import pandas as pd
from gensim.models import Word2Vec
import jieba
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as posg

In [77]:
# 停词表
# 停词表

def load_stop_word():
    stop_word = ['的', '。', '是', ' ', '\n', '日', '月', '.', '%','\u3000']
    stop_word_file = 'c:/data/stopwords'
    stop_word_dirs = os.listdir(stop_word_file)
    for textName in stop_word_dirs:
        filename = stop_word_file + '/' + textName
        with open(filename, 'r', encoding='utf-8') as file_object:
            line = file_object.readline()
            while line:
                stop_word.append(line[0:-1])
                line = file_object.readline()
    return stop_word
stop_word = load_stop_word()
print("停词表", stop_word[0:10])
#分词
 # 默认模式，分词的同时进行词性标注
# thu = thulac.thulac(filt=True, seg_only=True)
word_type = {}
def cut_doc(doc):
     # 默认模式，分词的同时进行词性标注
    
    words = posg.cut(doc)
    words_in_doc = ""
    for w in words:
        # print(word)
        if w.word in stop_word: continue
        words_in_doc +=  w.word + " "
        word_type[w.word] = w.flag
        
    return words_in_doc

words_in_doc = cut_doc("用来控制训练的文章数量")
words_in_doc,word_type

停词表 ['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '\u3000']


('控制 训练 文章 数量 ', {'控制': 'v', '训练': 'vn', '文章': 'n', '数量': 'n'})

In [78]:
doc_map = {}
label_map = {}
map = {}
DOC_COUNT_MAX = 20
wrong_num = []

def read_file_cut(doc_file_name, doc_id, label_index):
    with open(doc_file_name, 'r', encoding='utf-8') as file_object:
        doc_content = ""
        line = file_object.readline()
        while line:
            doc_content += line
            line = file_object.readline()
        wrong = 1
        for _ in range(0,4):
            try:
                doc_map[doc_id] = cut_doc(doc_content)
                wrong = 0
                break
            except Exception as e:
                continue
        if wrong == 1:
            wrong_num.append(doc_id)
        label_map[doc_id] = int(label_index)
    return 0

my_df = pd.DataFrame([],index=[],columns=['text','label'])

label_num = {}

with ThreadPoolExecutor(max_workers= int(10000)) as t:
    obj_list = []
    begin = time.time()
    
    news_file = 'c:/data/THUCNews'
    news_file_dirs = os.listdir(news_file)
    # print(news_file_dirs)
    
    label_index = -1
    for label in news_file_dirs:
        label_index += 1
        label_num[label] = label_index
        
        file = news_file + '/' + label
        doc_ids = os.listdir(file)
        doc_num = 0
        for doc_name in doc_ids:
            doc_num += 1
            if doc_num >= DOC_COUNT_MAX:
                break
                
            doc_file_name = file + '/' + doc_name
            doc_id = doc_name[0:-4]
            doc_content = ""
            
            
            obj = t.submit(read_file_cut, doc_file_name, doc_id, label_index)
            obj_list.append(obj)    
            
      
    for future in as_completed(obj_list):
        data = future.result()
    times = time.time() - begin
    print("多线程占据的全部时间"+str(times))
    print(len(obj_list))
    print(len(doc_map))
    print(len(label_map))
    
map['text'] = doc_map
map['label'] = label_map


my_df = pd.DataFrame(map)
train_df = my_df
print("错误的次数", len(wrong_num) )
print(train_df.shape)
train_df.to_csv("c:/data/news/train_data301.csv")

多线程占据的全部时间17.74009919166565
266
266
266
错误的次数 0
(266, 2)


In [85]:
train_df

,text,label
1000,揭秘 谢亚龙 带走 总局 电话 骗局 复制 南杨 轨迹 体坛周报 特约记者 张锐 北京 报道...,0
100000,姚明 未来 次节 出战 成疑 火箭 高层 称 改变 姚 战略 新浪 体育讯 北京 时间 11...,0
100007,普里 兹 比拉 恢复 训练 内线 板凳 匪徒 最迟 本月底 复出 新浪 体育讯 记者 周超 ...,0
100011,正 视频 直播 奇才 尼克斯 阿联 火拼 小斯 大将军 复出 新浪 体育讯 北京 时间 11...,0
0,马晓旭 意外 受伤 国奥 警惕 无奈 大雨 青睐 殷家 军 记者 傅亚雨 沈阳 报道 来到 ...,0
...,...,...
798982,2010 艺术品 秋 拍 上演 六宗 中国 艺术品 市场 经历 2008 年 低潮 2009...,13
798988,十二五 期间 钢铁业 重组 纵深 推进 □ 本报记者 钱唐 26 中国 钢铁产业 重组 发展...,13
798986,中 储粮 库存 化 7000 万吨 储粮 考验 国家 仓容 本报记者 杨 颢 吉林 上海 报...,13
798991,传 宝钢 炒 镍 亏损 9.9 亿元 宝钢 镍 价 赌局 21 世纪 经济 报道 记者 陈小...,13


In [86]:
# 基于 word2vec 生成 词向量
import numpy as np
useless = set()

doc_data = train_df['text']
common_text = []
for i in range(0, doc_data.size):
    doc = str(doc_data[i]).split(" ")
    common_text.append(doc)

# 词向量的维度，样本够的话300-500。
# min_count 最小次数计数，出现次数低于这个数的就不要了
# workers  线程数
# window 窗口大小
# sample 负例采样设置aa


vector_size = 500
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=4,
                  workers=12, sample=1e-5)
# model.save("word2vec.model")
vector = model.wv['中国']

# 词向量 合成 文本向量
doc_v_list = np.array([])

for split_doc in common_text:
    used_word = float(0.0)
    vector = vector * 0
    for word in split_doc:

        used_word += 1.0
        try:
            vector += model.wv[word]
        except Exception as e:
            # print(e.__traceback__.tb_frame.f_globals["__file__"])  # 发生异常所在的文件
            # print(e.__traceback__.tb_lineno)  # 发生异常所在的行数
            used_word -= 1.0

    vector /= used_word /200
    doc_v_list = np.append(doc_v_list, vector)

train_test = doc_v_list.reshape(len(common_text), vector_size)
train_test

array([[ 0.01300619,  0.07870772,  0.0751991 , ..., -0.01699982,
        -0.06281587,  0.06328117],
       [ 0.0176933 ,  0.04547055, -0.00361667, ..., -0.03124469,
        -0.04183216,  0.02868373],
       [-0.02646718,  0.02983414,  0.06214645, ...,  0.0084201 ,
        -0.06159022,  0.03181097],
       ...,
       [ 0.02134466,  0.00881413,  0.01832289, ..., -0.04710085,
        -0.01157468, -0.0209859 ],
       [ 0.00159464, -0.0288382 ,  0.02426349, ..., -0.02839828,
         0.00477065, -0.00726089],
       [ 0.01337562,  0.00715605,  0.04523276, ..., -0.02641956,
        -0.01911063, -0.00157576]])

In [108]:
# 基于每个文档的词语进行聚类

def get_key_word_from_doc(doc, model,ClusterNum = 6):
    #cut_doc(doc)
    used_word = 0
    word_matrix =  np.array([])
    # print(doc)
    index_of_word_map = {}
    doc = str(doc)
#     print(doc.split())
    for word in doc.split():
        if len(word) < 2: 
            continue
        try:
            word_matrix = np.append(word_matrix, model.wv[word])
            index_of_word_map[used_word] = word
            used_word += 1  
        except Exception as e:
            pass
        
    cluster_test = word_matrix.reshape(used_word, vector_size)
    ################### KMEANS #####################
    from sklearn.cluster import KMeans
    k_means =  KMeans(n_clusters=ClusterNum, random_state=9)
    y_pred = k_means.fit_predict(cluster_test)

    best_key_set = set()
    best_key_word_set = set()
    for center in k_means.cluster_centers_:
        # print(type(center))

        best_key = 0
        min_dist = np.sqrt(np.sum((center - cluster_test[0]) ** 2))
        
        for key in range(0, len(cluster_test)):
            if key in best_key_set: continue
            if index_of_word_map[key] in best_key_word_set: continue
            word_vector = cluster_test[key]
            #print(word_vector)
            dist = np.sqrt(np.sum((word_vector - center) ** 2))
            if dist < min_dist:
                #print(dist)
                best_key = key

        best_key_set.add(best_key)
        best_key_word_set.add(index_of_word_map[best_key])
        #print(best_key, index_of_word_map[best_key])
    
    return best_key_set, best_key_word_set,index_of_word_map
from snownlp import SnowNLP
for doc in train_df['text']:
    ClusterNum = 6
    print(get_key_word_from_doc(doc,model,ClusterNum)[1])
    s = SnowNLP(doc)
    print(s.keywords(ClusterNum))
    break

{'情况', '立案侦查', '16', '14', '正式', '揭秘'}
['谢', '亚龙', '总局', '侦查', '立案', '带走']


In [110]:


####################### TF-IDF #############

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score


tfidf = TfidfVectorizer(ngram_range=(1,4), max_features=300)
train_test = tfidf.fit_transform(train_df['text'])
train_test

<266x300 sparse matrix of type '<class 'numpy.float64'>'
	with 6532 stored elements in Compressed Sparse Row format>

In [89]:
sort = np.argsort(train_test.toarray(), axis = 1)[:, -10:]
names = tfidf.get_feature_names()
keyWords = pd.Index(names)[sort]
keyWords[0:21]

C:\Users\hys\AppData\Local\Temp\ipykernel_19916\4121189783.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  keyWords = pd.Index(names)[sort]


array([['这一', '位置', '27', '29', '事情', '一位', '25', '球队', '31', '球员'],
       ['这是', '未来', '采访', '比赛', '原因', '表现', '球队', '防守', '火箭', '姚明'],
       ['这是', '2009', '依然', '优势', '球队', '赛季', '时间', '表现', '篮板', '比赛'],
       ['采访', '游戏', '回答', '球员', '火箭', '记者', '11', '时间', '姚明', '球队'],
       ['很大', '一位', '找到', '不错', '一名', '防守', '特别', '调整', '这一', '球员'],
       ['只能', '对手', '回答', '进攻', '时间', '两个', '结束', '比赛', '火箭', '机会'],
       ['很大', '10', '压力', '26', '赛季', '12', '11', '情况', '主场', '球队'],
       ['感觉', '参加', '媒体', '状态', '面对', '球队', '详细', '100', '客场', '比赛'],
       ['新浪', '北京', '11', '这是', '北京 时间', '对手', '客场', '详细', '比赛', '主场'],
       ['几个', '情况', '20', '2008', '24', '2009', '管理', '球队', '赛季', '球员'],
       ['希望', '关注', '方式', '16', '参加', '真的', '依然', '优势', '近期', '一年'],
       ['冠军', '本场', '10', '12', '表现', '16', '17', '篮板', '球队', '比赛'],
       ['接受', '北京 时间', '第一', '不错', '采访', '这是', '表现', '一点', '球员', '球队'],
       ['时间', '最终', '只能', '机会', '表现', '球队', '选择', '比赛', '进攻', '防守'],
       ['记者', '美国', '

In [52]:
k_means.cluster_centers_

array([[ 0.08378126, -0.1839826 ,  0.16146725, ...,  0.02087184,
        -0.13922298, -0.30269699],
       [ 0.05630706, -0.12312471,  0.10766536, ...,  0.01407618,
        -0.09354932, -0.2032135 ],
       [ 0.02327915, -0.05024099,  0.04506252, ...,  0.00531093,
        -0.03750145, -0.08308181],
       [ 0.13345828, -0.29105409,  0.25545619, ...,  0.03428826,
        -0.22079782, -0.47747483]])

球员
190
不解
193
青睐
192
中
165


In [35]:
from gensim.models import Word2Vec

vector_size = 500
doc1 = '苹果 是 很好 的 手机'
common_text = []
common_text.append(doc1.split(" "))
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=0,
                  workers=2, sample=1e-5)
vec1 = model.wv['苹果']
vec1 

array([ 5.14035230e-04,  1.69305800e-04, -5.07888326e-04,  1.87153334e-03,
        5.51501755e-04,  8.19129462e-04, -2.36587046e-04,  1.80900577e-04,
        1.32486154e-03, -1.45300393e-04,  6.68301596e-04, -1.34404181e-04,
        1.04967924e-03,  7.27986800e-04,  5.16505737e-04, -1.06197887e-03,
       -9.41620325e-04,  8.61604232e-04, -1.18143414e-03, -3.60412596e-05,
       -1.26703264e-04,  6.98648451e-04, -1.68845325e-03,  1.76344684e-03,
       -2.89749150e-04, -1.06618542e-03,  8.10893544e-04, -3.87098320e-04,
       -1.55309681e-03, -8.99359700e-04, -7.75456429e-05, -1.78929477e-03,
        1.14112379e-04,  4.88358492e-04, -6.45054795e-04,  5.14080049e-04,
        4.96159540e-04,  1.99764292e-03,  2.86100869e-04,  4.04107559e-04,
        5.55912033e-04, -4.15574556e-04, -1.73972326e-03,  1.60463480e-03,
       -3.94939416e-04, -1.93846272e-03, -1.31040101e-03, -7.89172191e-04,
        7.90869235e-04,  1.00771617e-03,  1.21720217e-03, -1.35405210e-03,
        1.37767318e-04, -

In [36]:

doc2 = '多 吃 苹果 有益 健康 是 很好 的 水果'
common_text = []
common_text.append(doc2.split(" "))
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=0,
                  workers=2, sample=1e-5)
vec2 = model.wv['苹果']
vec2

array([ 3.8861752e-05,  4.2906712e-04,  2.1312857e-04,  1.5173388e-03,
       -8.8433886e-04,  6.4919470e-05,  1.4013071e-03,  4.5035887e-04,
        8.4854983e-04,  8.6065533e-04, -2.5312949e-04, -1.6645150e-03,
        6.0885097e-04, -1.7746286e-03, -5.3217221e-04, -1.8784089e-03,
       -9.2403410e-05, -1.5726585e-03,  1.4827952e-03, -9.7806356e-04,
        1.9836598e-03, -1.8064532e-03, -8.6243532e-04,  3.8578510e-04,
       -5.9948635e-04,  2.8915645e-04, -7.3616888e-04, -1.7883444e-04,
       -1.1454734e-03, -1.9642028e-03,  1.2678395e-03,  7.6730728e-05,
        1.3019033e-03, -1.4945259e-03,  7.3270989e-04, -1.9795776e-03,
        1.5221463e-03,  1.1849094e-03, -1.2234959e-03,  1.6426119e-03,
       -1.1160097e-03, -1.7107200e-03,  3.0005694e-04,  6.3599250e-04,
       -1.6193204e-03, -1.7868528e-03,  1.8342643e-03,  3.2875489e-04,
        1.7486524e-04,  4.9564650e-04, -7.0142891e-04,  3.4700776e-04,
        3.9161634e-04,  1.9612003e-03, -1.1216264e-03,  4.5460509e-04,
      

In [28]:
import numpy as np

def euclidean(x, y):
    return np.sqrt(np.sum((vec1 - vec2)**2))

('控制 训练 文章 数量 ', {'控制': 'v', '训练': 'vn', '文章': 'n', '数量': 'n'})

In [37]:
str3 = '苹果公司1980年12月12日公开招股上市，2012年创下6235亿美元的市值记录，截至2014年6月，苹果公司已经连续三年成为全球市值最大公司。当地时间2020年8月19日，苹果公司市值首次突破2万亿美元。 [1]  苹果公司在2016年世界500强排行榜中排名第9名。 [2]  2013年9月30日，在宏盟集团的“全球最佳品牌”报告中，苹果公司超过可口可乐成为世界最有价值品牌。2014年，苹果品牌超越谷歌（Google），成为世界最具价值品牌。2021年《财富》世界500强排行榜第6名。 [125]  北京时间2022年1月4日凌晨2点45分左右，美国科技巨头苹果的股价达到了182.88美元，市值第一次站上了三万亿美元的台阶，这不仅是全球首个3万亿市值，也相当于全球第五大经济体的GDP体量，仅次于美国、中国、日本及德国'
doc3 = cut_doc(str3)


common_text = []
common_text.append(doc3.split(" "))
model = Word2Vec(sentences=common_text, vector_size=vector_size, window=5, min_count=0,
                  workers=2, sample=1e-5)
vec3 = model.wv['苹果']
vec1,vec2,vec3


(array([ 5.14035230e-04,  1.69305800e-04, -5.07888326e-04,  1.87153334e-03,
         5.51501755e-04,  8.19129462e-04, -2.36587046e-04,  1.80900577e-04,
         1.32486154e-03, -1.45300393e-04,  6.68301596e-04, -1.34404181e-04,
         1.04967924e-03,  7.27986800e-04,  5.16505737e-04, -1.06197887e-03,
        -9.41620325e-04,  8.61604232e-04, -1.18143414e-03, -3.60412596e-05,
        -1.26703264e-04,  6.98648451e-04, -1.68845325e-03,  1.76344684e-03,
        -2.89749150e-04, -1.06618542e-03,  8.10893544e-04, -3.87098320e-04,
        -1.55309681e-03, -8.99359700e-04, -7.75456429e-05, -1.78929477e-03,
         1.14112379e-04,  4.88358492e-04, -6.45054795e-04,  5.14080049e-04,
         4.96159540e-04,  1.99764292e-03,  2.86100869e-04,  4.04107559e-04,
         5.55912033e-04, -4.15574556e-04, -1.73972326e-03,  1.60463480e-03,
        -3.94939416e-04, -1.93846272e-03, -1.31040101e-03, -7.89172191e-04,
         7.90869235e-04,  1.00771617e-03,  1.21720217e-03, -1.35405210e-03,
         1.3

In [38]:
print(euclidean(vec1, vec2)-euclidean(vec1,vec3))

0.0


In [39]:
print(euclidean(vec1, vec2 ))

0.036175095


In [40]:
print(euclidean(vec1,vec3))

0.036175095


In [114]:
from transformers import BertTokenizer,BertModel
from pprint import pprint
from transformers import BertModel

vocab_file = '../aliYunCls/BertModel/bert-base-chinese-vocab.txt'
tokenizer = BertTokenizer(vocab_file)
model = BertModel.from_pretrained('../aliYunCls/BertModel/bert-base-chinese')
    
def word_to_vec(word, sequence):
     # sequence 待分词的句子
     # sequence = "苹果好的手机" 
    sequence = str(sequence)
    tokenized_sequence = tokenizer.tokenize(sequence)  # 调用tokenize方法进行分词
    #print(tokenized_sequence)  # 输出
    inputs = tokenizer([sequence],return_tensors='pt')
    encoded_sequence = inputs["input_ids"] 
    #pprint(inputs)
    out = model(**inputs)
    x = out['last_hidden_state'][0][0]
    x = x * 0
    for id in word_id:
        x = x + out['last_hidden_state'][0][id]  
        
    return x
doc1 = '苹果 是 很好 的 手机'
vec1 = word_to_vec([1,2], doc1)
print(vec1.shape)

Some weights of the model checkpoint at ../aliYunCls/BertModel/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 768])


In [73]:
doc1 = '苹果 是 很好 的 手机'
vec1 = word_to_vec([1,2], doc1)

doc2 = '多 吃 苹果 有益 健康 是 很好 的 水果'
vec2 = word_to_vec([3,4], doc2)

str3 = '苹果公司1980年12月12日公开招股上市，2012年创下6235亿美元的市值记录，截至2014年6月，苹果公司已经连续三年成为全球市值最大公司。当地时间2020年8月19日，苹果公司市值首次突破2万亿美元。 [1]  苹果公司在2016年世界500强排行榜中排名第9名。 [2]  2013年9月30日，在宏盟集团的“全球最佳品牌”报告中，苹果公司超过可口可乐成为世界最有价值品牌。2014年，苹果品牌超越谷歌（Google），成为世界最具价值品牌。2021年《财富》世界500强排行榜第6名。 [125]  北京时间2022年1月4日凌晨2点45分左右，美国科技巨头苹果的股价达到了182.88美元，市值第一次站上了三万亿美元的台阶，这不仅是全球首个3万亿市值，也相当于全球第五大经济体的GDP体量，仅次于美国、中国、日本及德国'
doc3 = cut_doc(str3)
vec3 = word_to_vec([1,2], doc3)

Some weights of the model checkpoint at ../aliYunCls/BertModel/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ../aliYunCls/BertModel/bert-base-chinese were not used when initializing BertModel: ['cls.pr

In [54]:
# print(euclidean(vec1, vec2)-euclidean(vec1,vec3))
print(type(vec1))

<class 'torch.Tensor'>


In [67]:
import torch
def dis(a,b):    
    return torch.sqrt(((a-b)**2).sum())


In [68]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # **运算符是求幂运算
x-y,((x-y)**2).sum(),dis(x,y)

(tensor([-1.,  0.,  2.,  6.]), tensor(41.), tensor(6.4031))

In [75]:
dis(vec3,vec2) ,dis(vec1,vec3)

(tensor(44.2874, grad_fn=<SqrtBackward>),
 tensor(31.0007, grad_fn=<SqrtBackward>))